In [96]:
from datetime import datetime
import requests
from algosdk import future, encoding, logic
from contracts.crowdfounding import utils
from algosdk.v2client import algod



In [15]:
# Test data

private_key = 'ixz3gYNh1EsJszFCgUgrSQlT//fuGZK7hAzdNftlVRZ5rzuM7Zz5tPXLhPWRvpvNYsBwrvyQt+HCyJstOgiSjg=='
address = 'PGXTXDHNTT43J5OLQT2ZDPU3ZVRMA4FO7SILPYOCZCNS2OQISKHIUGJ3SM'


# address need to be founded from the sandbox
money_address = "6B2KSSG2ZWW3X3NNONJ4NNFWJXBHGI3K7VUZHISFKUVP2UEXF22QJDF7WY"
amount = 100000000
print("goal clerk send -a " + str(amount) + " -f " + money_address + " -t " + address)


goal clerk send -a 100000000 -f 6B2KSSG2ZWW3X3NNONJ4NNFWJXBHGI3K7VUZHISFKUVP2UEXF22QJDF7WY -t PGXTXDHNTT43J5OLQT2ZDPU3ZVRMA4FO7SILPYOCZCNS2OQISKHIUGJ3SM


In [84]:
query = {'address': address}
response = requests.get('http://localhost:8501/address_info', params=query)
print(response.json())


{'address': 'PGXTXDHNTT43J5OLQT2ZDPU3ZVRMA4FO7SILPYOCZCNS2OQISKHIUGJ3SM', 'amount': 100000691, 'amount-without-pending-rewards': 99999800, 'apps-local-state': [], 'apps-total-schema': {'num-byte-slice': 2, 'num-uint': 3}, 'assets': [], 'created-apps': [{'id': 60, 'params': {'approval-program': 'BiADAQADMRgjEkAAUTEZIxI2GgCABXNldHVwEhBAAAEAMwAQIhIzARCBBhIQRLEkshCBwIQ9siIksiMisiSABHVuaXSyJTIHFrImMgqyKTIKsioyCrIrMgqyLLMiQzEbgQQSNhoAFyMNEDIHNhoBFwwQMgc2GgIXDBA2GgEXNhoCFwwQRIAHZm91bmRlcjEAZ4AFc3RhcnQ2GgEXZ4ADZW5kNhoCF2eAC3Bvb2xfdGFyZ2V0NhoAF2eADWZvdW5kaW5nX25hbWU2GgNnIkM=', 'clear-state-program': 'BoEBQw==', 'creator': 'PGXTXDHNTT43J5OLQT2ZDPU3ZVRMA4FO7SILPYOCZCNS2OQISKHIUGJ3SM', 'global-state': [{'key': 'Zm91bmRlcg==', 'value': {'bytes': 'ea87jO2c+bT1y4T1kb6bzWLAcK78kLfhwsibLToIko4=', 'type': 1, 'uint': 0}}, {'key': 'ZW5k', 'value': {'bytes': '', 'type': 2, 'uint': 1653861600}}, {'key': 'c3RhcnQ=', 'value': {'bytes': '', 'type': 2, 'uint': 1653775200}}, {'key': 'Zm91bmRpbmdfbmFtZQ==', 'valu

In [37]:
# This cell is for testing purpose only

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)

### Creating the contract

API EXAMPLE

 > http://localhost:8501/create_contract?sender=SUBSTITUTE_WITH_PUBLIC_KEY_&pool_name=prova&target=10000&startTime=1653775200&endTime=1653948000


In [33]:
startTime = int(datetime.timestamp(datetime(2022,5,29)))
endTime = int(datetime.timestamp(datetime(2022,5,30)))
pool_name = "prova_pmz"
target = 1000000 # microAlgo


query = {"sender": address, "pool_name":pool_name,"target":target, "startTime": startTime, "endTime":endTime}
txt_to_sign_encoded = requests.get('http://localhost:8501/create_contract', params=query).json()

In [43]:
txt_to_sign = encoding.future_msgpack_decode(txt_to_sign_encoded)

signedTxn = txt_to_sign.sign(private_key)

txid = algod_client.send_transaction(signedTxn)
return_conf = utils.inspect_transaction(txid,algod_client)

Transaction information: {
    "application-index": 60,
    "confirmed-round": 11250,
    "global-state-delta": [
        {
            "key": "Zm91bmRpbmdfbmFtZQ==",
            "value": {
                "action": 1,
                "bytes": "cHJvdmFfcG16"
            }
        },
        {
            "key": "cG9vbF90YXJnZXQ=",
            "value": {
                "action": 2,
                "uint": 1000000
            }
        },
        {
            "key": "c3RhcnQ=",
            "value": {
                "action": 2,
                "uint": 1653775200
            }
        },
        {
            "key": "ZW5k",
            "value": {
                "action": 2,
                "uint": 1653861600
            }
        },
        {
            "key": "Zm91bmRlcg==",
            "value": {
                "action": 1,
                "bytes": "ea87jO2c+bT1y4T1kb6bzWLAcK78kLfhwsibLToIko4="
            }
        }
    ],
    "pool-error": "",
    "sender-rewards": 800,
    "tx

### SETUP THE CONTRACT

In [95]:
import json
import pprint

query = {'address': address}
response = requests.get('http://localhost:8501/address_info', params=query)

result_dict = response.json()
lista_app_create_account = result_dict['created-apps']

id = logic.get_application_address(lista_app_create_account[0]['id']) # questo è l'id della nostra applicazione appena creata 

In [90]:
from algosdk.constants import MIN_TXN_FEE


parameters = algod_client.suggested_params()
parameters.flat_fee = True
parameters.fee = 4 * MIN_TXN_FEE



fundAppTxn = future.transaction.PaymentTxn(
        sender = address,
        receiver = logic.get_application_address(id),
        amt = 500000,
        sp = algod_client.suggested_params(),
    )


setupTxn = future.transaction.ApplicationNoOpTxn(
        sender = address,
        index=id,
        app_args=[b"setup"],
        sp=algod_client.suggested_params()
)

future.transaction.assign_group_id([fundAppTxn, setupTxn])


signedFundAppTxn = fundAppTxn.sign(private_key)
signedSetupTxn = setupTxn.sign(private_key)

indedd = algod_client.send_transactions([signedFundAppTxn, signedSetupTxn])
utils.inspect_transaction(indedd, algod_client)